In [2]:
from os import listdir
from os.path import isfile, join
import pandas as pd
import plotly.express as px
import os
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import requests
import re
from django.utils import timezone


from urllib.request import urlopen
from bs4 import BeautifulSoup
from numpy import full
from datetime import datetime
pd.options.mode.chained_assignment = None 
import time

import aiohttp
import asyncio
import lxml.html

In [3]:

def catch_links_all_pages(url):
    oglasi = []
    page1 = requests.get(url)
    soup1 = BeautifulSoup(page1.content, "html.parser")

    #Extract the number of pages from navigation bar
    page_nav_class = soup1.find_all('ul', attrs={'class':"pagination"}) 
    page_nav = page_nav_class[0].find_all('li')
    
    
    # #Take the last navigation element
    # last = page_nav[-1]
    # #Extract the number and clean data
    # page_number = last.find('a').get('href')[-3:]
    # page_number = re.sub("[^0-9]", "", page_number)
    # fix = "&num="
    
    pages = []

    #Check if there are multiple pages in the navigation
    if not page_nav:

        pages.append(url)

    else:
        #Take the last navigation element
        last = page_nav[-1]
        #Extract the number and clean data
        page_number = last.find('a').get('href')[-3:]
        page_number = re.sub("[^0-9]", "", page_number)
        fix = "&num="
        # print(page_number)

        #Create a list of all pages with links
        pages.append(url)
        for i in range(2,int(page_number)+1):
                link = url + fix + str(i)
                pages.append(link)

        pages = list(set(pages))
       
    #Scrape all resluts from every page
    for page in pages:        
        web_page = requests.get(page)
        soup1 = BeautifulSoup(web_page.content, "html.parser")
        results = soup1.find_all('div', attrs={'class':"OglasiRezHolder"}) 
        for result in results:
            try:
                link = result.find('a').get('href')
                oglasi.append(link)
            except:
                pass

    oglasi = list(set(oglasi))

    return oglasi
    
async def data_dict_index(url_list):
    data_list = []
    errors = []

    async with aiohttp.ClientSession() as session:
        for url in url_list:
            try:
                async with session.get(url) as response:
                    html = await response.text()
                    doc = lxml.html.fromstring(html)

                    my_dict = {}
                    for row in doc.xpath("//li[@class='labela']"):
                        a = row.text_content()
                        b = row.getnext().text_content().replace("\r\n", "")
                        if a in my_dict.keys():
                            if b < my_dict[a]:
                                my_dict[a] = b
                        else:
                            my_dict[a] = b
                    
                    my_dict["Cijena"] = doc.xpath("//div[@class='price']/span")[0].text_content()
                    my_dict["Link"] = url

                    datum = doc.xpath("//div[@class='published']")
                    date = datum[1].text_content()
                    my_dict["Datum"] = date

                    data_list.append(my_dict)

            except:
                errors.append(url)

    return data_list

def dataframe_cleaner(df):
    #df = pd.DataFrame(data_list)

    #Data Cleaning

    #Drop rows w/o sqm, convert sqm to float
    df = df[df['Stambena površina u m2'].notna()]
    df["Stambena površina u m2"] = df["Stambena površina u m2"].str.replace(",",".")
    df["Stambena površina u m2"] = df["Stambena površina u m2"].astype('float')

    #Convert price to float, remove no price rows, rempve unrealistic low prices
    df["Cijena"] = df["Cijena"].str.replace("€","")

    df["Cijena"] = df["Cijena"].str.replace(".","")
    df["Cijena"] = df["Cijena"].str.replace(",",".")

    df["Cijena"] = df["Cijena"].str.replace(" ","")

    df = df[df["Cijena"] != "0,00 "]
    df["Cijena"] = df["Cijena"].astype('float')
    df = df[df["Cijena"] > 10000]

    #Replace Nan with "No info"
    df['Godina izgradnje'].fillna('No INFO', inplace=True)

    #Remove where m2 == 0
    df = df[df["Stambena površina u m2"] > 10]
    df = df[df["Stambena površina u m2"] < 10000]




    #Add a price per sqm meter column
    df["€/m²"] = round(df["Cijena"] / df["Stambena površina u m2"],2)

    #Convert Datum to datetime
    df["Datum"] = pd.to_datetime(df["Datum"], format="%d.%m.%Y")

    #Calculate days online
    df['Days Online'] = ( pd.Timestamp('now') - df['Datum']).dt.days

    return df

async def main():
    url_list = [
        "http://example.com/1",
        "http://example.com/2",
        "http://example.com/3",
    ]
    data_list = await data_dict_index(url_list)
    return data_list

loop = asyncio.get_event_loop()
loop.run_until_complete(main())
loop.close()

In [6]:

# get the start time
st = time.time()


url = "https://www.index.hr/oglasi/prodaja-stanova/gid/3278?pojamZup=1166&tipoglasa=1&sortby=1&elementsNum=100&city=1638&naselje=0&cijenaod=0&cijenado=21000000&vezani_na=988-887_562-563_978-1334&num=1"
URL_rovinj = "https://www.index.hr/oglasi/prodaja-stanova/gid/3278?pojam=&sortby=1&elementsNum=100&cijenaod=0&cijenado=21000000&tipoglasa=1&pojamZup=1154&city=1294&naselje=3374&attr_Int_988=&attr_Int_887=&attr_bit_stan=&attr_bit_brojEtaza=&attr_gr_93_1=&attr_gr_93_2=&attr_Int_978=&attr_Int_1334=&attr_bit_eneregetskiCertifikat=&vezani_na=988-887_562-563_978-1334"
URL_zg = "https://www.index.hr/oglasi/prodaja-stanova/gid/3278?pojam=&sortby=1&elementsNum=100&cijenaod=0&cijenado=21000000&tipoglasa=1&pojamZup=1153&city=&naselje=&attr_Int_988=&attr_Int_887=&attr_bit_stan=&attr_bit_brojEtaza=&attr_gr_93_1=&attr_gr_93_2=&attr_Int_978=&attr_Int_1334=&attr_bit_eneregetskiCertifikat=&vezani_na=988-887_562-563_978-1334"


url = URL_zg
city_name = "Zagreb"
ime = "_zagreb"



#find all pages
pages = catch_links_all_pages(url)

# get the end time
et = time.time()

# get the execution time
elapsed_time = et - st
elapsed_time_min = elapsed_time/60
print('catch_links_all_pages', elapsed_time_min, 'minutes')


#scrape all pages
data_list = []

data_list = data_dict_index(pages)

# get the end time
et = time.time()

# get the execution time
elapsed_time = et - st
elapsed_time_min = elapsed_time/60
print('data_dict_index:', elapsed_time_min, 'minutes')

print(data_list)


# #create dataframe
# df = pd.DataFrame(data_list)
# current_date = datetime.now().date()

# #check raw entries, save raw dataframe
# dj_raw_entries = len(df.index)

# full_ime_raw = str(current_date) + "_RAW" + ime + ".xlsx"
# dj_df_raw  = df.to_excel(full_ime_raw)

# #clean dataframe
# df = dataframe_cleaner(df)

# # get the end time
# et = time.time()

# # get the execution time
# elapsed_time = et - st
# elapsed_time_min = elapsed_time/60
# print('dataframe_cleaner', elapsed_time_min, 'minutes')

# full_ime =  str(current_date) + ime + ".xlsx"

# dj_df = df.to_excel(full_ime)
# dj_clean_entries = len(df.index)

# dj_avg_price_sqrm = round(df["€/m²"].mean(),2)
# dj_avg_size = round(df["Stambena površina u m2"].mean(),2)
# dj_avg_year = df[df["Godina izgradnje"] != "No INFO"]["Godina izgradnje"].mode()[0]



# max_df = df[df['Cijena'] == df['Cijena'].max()]
# max = [max_df["Cijena"].values[0], max_df["Link"].values[0]]

# max_per_sqr_df = df[df['€/m²'] == df['€/m²'].max()]
# max_per_sqr = [max_per_sqr_df['€/m²'].values[0], max_per_sqr_df['Link'].values[0]]

# min_df = df[df['Cijena'] == df['Cijena'].min()]
# min = [min_df["Cijena"].values[0], min_df["Link"].values[0]]

# min_per_sqr_df = df[df['€/m²'] == df['€/m²'].min()]
# min_per_sqr = [min_per_sqr_df['€/m²'].values[0], min_per_sqr_df['Link'].values[0]]
# med_sale_price = df["Cijena"].mean()

# # time_now = timezone.now()

# #     new_entry = Document(
# #     document = dj_df, document_raw = dj_df_raw, uploaded_at = time_now,
# #     calendar_week = time_now.isocalendar().week , raw_entries = dj_raw_entries, clean_entries = dj_clean_entries,
# #     city = city_name, avg_price_sqrm = dj_avg_price_sqrm, avg_size = dj_avg_size,
# #     avg_year = dj_avg_year, highest_price = max[0], highest_price_link = max[1],
# #     highest_price_sqrm = max_per_sqr[0], highest_price_sqrm_link = max_per_sqr[1],
# #     lowest_price = min[0], lowest_price_link = min[1], lowest_price_sqrm = min_per_sqr[0],
# #     lowest_price_sqrm_link = min_per_sqr[1]
# # )




# # get the end time
# et = time.time()

# # get the execution time
# elapsed_time = et - st
# elapsed_time_min = elapsed_time/60
# print('Execution time:', elapsed_time_min, 'minutes')


catch_links_all_pages 0.7498210748036702 minutes
data_dict_index: 0.7498210748036702 minutes
<coroutine object data_dict_index at 0x00000199250CABC0>


In [ ]:
def data_dict_index(url_list):

    data_list = []
    errors = []

    for url in url_list:
        try:

            page = requests.get(url)    

            soup = BeautifulSoup(page.content, "html.parser")

            my_dict = {}
            osnovno = soup.find_all('li', attrs={'class':'labela'})
            for row in osnovno:
                a = row.text
                b = row.findNextSibling().get_text().replace("\r\n", "")

                if a in my_dict.keys():
                    if b < my_dict[a]:
                        my_dict[a] = b
                    else:
                        pass
                else:
                    my_dict[a] = b
            
            my_dict["Cijena"] = soup.find('div', attrs={'class':'price'}).find('span').text
            my_dict["Link"] = url

            datum = soup.find_all('div', attrs={'class':'published'})
            regex = r"\d{2}.\d{2}.\d{4}"
            date = re.findall(regex,datum[1].get_text())[0]
            my_dict["Datum"] = date

            data_list.append(data_dict_index(page))

        except:
            errors.append(url)

    return data_list